In [50]:
# Paso 1: Importación de Librerías
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [11]:
# Paso 2: Cargar el Modelo BERT
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
model = BertModel.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
# Paso 3: Función para Obtener Embeddings de BERT
def bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    outputs = model(**inputs)
    # Tomar la media a lo largo de la dimensión de la secuencia para reducir a 2D
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings


In [13]:
# Paso 4: Reemplazo del Código de Chunking
def load_documents(file_paths):
    documents = []
    doc_names = []
    for name, path in file_paths.items():
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()
            doc = {'page_content': content}
            documents.append(doc)
            doc_names.append(name)
    return documents, doc_names

def split_text(documents):
    chunk_size = 500
    chunk_overlap = 60
    chunks = []
    doc_indices = []
    for i, doc in enumerate(documents):
        content = doc['page_content']
        start = 0
        while start < len(content):
            end = start + chunk_size
            if end > len(content):
                end = len(content)
            chunks.append({'page_content': content[start:end]})
            start += (chunk_size - chunk_overlap)
            doc_indices.append(i)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
    return chunks, doc_indices

# Función para reducir el tamaño del texto en el hover y permitir saltos de línea
def format_chunk(chunk, max_length=50):
    words = chunk.split()
    formatted_chunk = ""
    line_length = 0
    for word in words:
        if line_length + len(word) <= max_length:
            formatted_chunk += word + " "
            line_length += len(word) + 1
        else:
            formatted_chunk += "<br>" + word + " "
            line_length = len(word) + 1
    return formatted_chunk

# Función para encontrar los chunks más similares a las preguntas
def find_most_similar_chunks(question_embeddings, tfidf_matrix, top_k=3):
    similarities = cosine_similarity(question_embeddings, tfidf_matrix)
    top_indices = similarities.argsort(axis=1)[:, -top_k:]
    top_similarities = np.array([similarities[i, indices] for i, indices in enumerate(top_indices)])
    return top_indices, top_similarities


In [14]:
# Paso 5: Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Paso 6: Cargar Documentos
file_paths = {
    "Auditorias": "/content/drive/MyDrive/PROYECTO TFM (team NF-AI)/CODIGO/txt/auditorias.txt",
    "Ausentismo": "/content/drive/MyDrive/PROYECTO TFM (team NF-AI)/CODIGO/txt/ausentismo.txt",
    "Estructura Organizativa": "/content/drive/MyDrive/PROYECTO TFM (team NF-AI)/CODIGO/txt/estructura organizativa.txt",
    "Riesgos": "/content/drive/MyDrive/PROYECTO TFM (team NF-AI)/CODIGO/txt/riesgos.txt",
    "Informacion": "/content/drive/MyDrive/PROYECTO TFM (team NF-AI)/CODIGO/txt/información.txt",
    "Emergencia": "/content/drive/MyDrive/PROYECTO TFM (team NF-AI)/CODIGO/txt/emergencia.txt"
}


In [34]:
# Paso 7: Lista de Preguntas
questions = [
    "¿Cómo se diferencia la ubicación física de la operativa?",  # Manual Estructura Organizativa pág. 60 y 73.
    "¿Qué información se incluye en el informe de la evaluación de riesgos que ofrece la plataforma y cuál es su utilidad para la gestión de riesgos en la empresa?",  # Manual Identificación y Evaluación de Riesgos (IER); pág. 24 - 26.
    "¿Hay indicadores relativos al cumplimiento de normas?",  # Manual Auditorías pág. 16 17 30 y 31.
    "¿Qué papel juega la estructura organizativa en la funcionalidad general de la plataforma y cómo interactúa con otros módulos?",  # Manual Estructura Organizativa
    "¿Qué permite realizar la evaluación de riesgos?",  # Manual Identificación y Evaluación de Riesgos (IER)
    "¿Qué tipo de documentos se pueden almacenar y compartir en el espacio denominado 'Documentos' y cuál es su importancia dentro del contexto de la gestión empresarial en la plataforma?",  # Manual del Repositorio documental
    "¿Se pueden llevar a cabo auditorías internas?",  # Manual Auditorías
    "¿Cuál es la tasa de adopción de esta plataforma en el mercado?",  # Pregunta que no se puede resolver con los manuales
    "¿Cómo se compara esta plataforma con otras soluciones de gestión de SST?",  # Pregunta que no se puede resolver con los manuales
    "¿Qué día hará mañana?"  # Pregunta totalmente fuera del contexto
]


In [33]:
documents, doc_names = load_documents(file_paths)
chunks, doc_indices = split_text(documents)

all_chunks = [chunk['page_content'] for chunk in chunks]
chunk_to_manual = [doc_names[i] for i in doc_indices]

Split 6 documents into 209 chunks.


In [41]:
chunk_embeddings = np.array([bert_embeddings(chunk) for chunk in all_chunks]).squeeze()
question_embeddings = np.array([bert_embeddings(question) for question in questions])



Shape of chunk_embeddings: (209, 768)
Shape of question_embeddings: (10, 1, 768)


In [55]:
# Verificar y ajustar la forma de los embeddings
if len(chunk_embeddings.shape) == 3:
    chunk_embeddings = chunk_embeddings.reshape(chunk_embeddings.shape[0], -1)
if len(question_embeddings.shape) == 3:
    question_embeddings = question_embeddings.reshape(question_embeddings.shape[0], -1)

print(f"Shape of chunk_embeddings: {chunk_embeddings.shape}")
print(f"Shape of question_embeddings: {question_embeddings.shape}")


Shape of chunk_embeddings: (209, 768)
Shape of question_embeddings: (10, 768)


In [56]:
# Paso 8: Reducción de Dimensionalidad
pca = PCA(n_components=50)
pca_result = pca.fit_transform(chunk_embeddings)

tsne = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_result = tsne.fit_transform(pca_result)

df = pd.DataFrame(tsne_result, columns=['x', 'y'])
df['manual'] = chunk_to_manual
df['chunk'] = all_chunks


### CON euclidean_distances

In [57]:
# Función get_similar_chunks para usar distancia euclidiana
def get_similar_chunks(question_index, top_k=3):
    distances = euclidean_distances([question_embeddings[question_index]], chunk_embeddings)[0]
    top_indices = distances.argsort()[:top_k]  # Menores distancias
    return [(all_chunks[i], distances[i]) for i in top_indices]

In [58]:
# Paso 10: Obtener y Mostrar los Chunks Más Similares
for i, question in enumerate(questions):
    print(f"Question: {question}")
    similar_chunks = get_similar_chunks(i)
    for chunk, distance in similar_chunks:
        print(f"Euclidean Distance: {distance}")
        print(f"Chunk: {chunk}")
        print("-" * 50)

Question: ¿Cómo se diferencia la ubicación física de la operativa?
Euclidean Distance: 12.849054336547852
Chunk: eacción tipo emergencia   instalación actual reacción frente principal amenaza identificado .

5   •   Establecer mantener esquema organización interno , confeccionar   equipo respuesta intervención frente emergencia .   •   minimizar pérdida material ambiental derivado   situación emergencia poder presentar .   •   Minimizar riesgo desencadenar emergencia , control   .    •   Restablecer ritmo normal trabajo brevedad .   ¿ SIMULACRO ?

simulacro ensayo ejercicio adiestramiento práctico actuar c
--------------------------------------------------
Euclidean Distance: 12.87404727935791
Chunk: distinto ubicación   empresa ( ubicación físico operativo ) .      herencia funcionar distinto depender provenir ubicación físico   operativo . caso , herencia seguir dirección “ abajo ” .   , nivel alto estructura organizativo

nivel alto estructura organizativo nivel bajo .   contrario ,

### CON cosine_similartity

In [59]:
# Función get_similar_chunks para usar similitud del coseno
def get_similar_chunks(question_index, top_k=3):
    similarities = cosine_similarity([question_embeddings[question_index]], chunk_embeddings)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [(all_chunks[i], similarities[i]) for i in top_indices]

# Paso 10: Obtener y Mostrar los Chunks Más Similares
for i, question in enumerate(questions):
    print(f"Question: {question}")
    similar_chunks = get_similar_chunks(i)
    for chunk, similarity in similar_chunks:
        print(f"Cosine Similarity: {similarity}")
        print(f"Chunk: {chunk}")
        print("-" * 50)

Question: ¿Cómo se diferencia la ubicación física de la operativa?
Cosine Similarity: 0.6616424918174744
Chunk: 
simulacro ensayo ejercicio adiestramiento práctico actuar caso   emergencia , cuyo realización permitir comprobar real adecuación   previsto Plan emergencia organización necesidad existente servir ,   mayoría ocasión , identificar cabo actividad mejora   procedimiento establecido medio instalación .   simulacro emergencia ejercicio práctico

simulacro emergencia ejercicio práctico implicar participación   personal caso involucrar medio externo poder   bombero , protección civil , policía person
--------------------------------------------------
Cosine Similarity: 0.6599546670913696
Chunk: Versión   Fecha   Comentarios   Autor   1.0   29/09/2022   Redacción   Núria Zaragoza

Sabentis plataforma gestión especializado ámbito Seguridad Salud   Trabajo adaptado tecnología modelo gestión   prevención .    plataforma distinto tipo auditoría objetivo , figurar   evaluación Estándare

### Comprobación de embeddings con Bert

In [45]:
# Definir tres frases: dos similares y una distinta
phrase_1 = "El sol brilla intensamente en el cielo azul."
phrase_2 = "El cielo está despejado y el sol brilla."
phrase_3 = "El coche rojo está estacionado en la calle."

# Tokenizar y obtener embeddings para cada frase
phrases = [phrase_1, phrase_2, phrase_3]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embedding(phrase):
    inputs = tokenizer(phrase, return_tensors='pt')
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

embeddings = [get_embedding(phrase) for phrase in phrases]

# Calcular distancias entre embeddings
dist_12 = np.linalg.norm(embeddings[0] - embeddings[1])
dist_13 = np.linalg.norm(embeddings[0] - embeddings[2])
dist_23 = np.linalg.norm(embeddings[1] - embeddings[2])

print(f"Distancia entre frases similares (1 y 2): {dist_12}")
print(f"Distancia entre frase distinta (1 y 3): {dist_13}")
print(f"Distancia entre frase distinta (2 y 3): {dist_23}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Distancia entre frases similares (1 y 2): 4.958994388580322
Distancia entre frase distinta (1 y 3): 5.729472637176514
Distancia entre frase distinta (2 y 3): 3.9385998249053955


Al analizar los resultados obtenidos al calcular las distancias entre los embeddings de las frases usando BERT, puedo hacer las siguientes observaciones:  


**Entre las frases 1 y 2**, que son similares, la distancia es de 4.958994388580322. Esto muestra que el modelo ha identificado cierta similitud semántica entre ambas, lo que es esperado dado que ambas frases describen un cielo despejado y el sol brillando.  


**Entre las frases 1 y 3**, que son conceptualmente diferentes, la distancia es mayor, de 5.729472637176514, indicando que el modelo reconoce adecuadamente que la frase sobre el coche rojo estacionado es distinta a la que habla del sol y el cielo.  


**Entre las frases 2 y 3**, el resultado es inesperado. La distancia de 3.9385998249053955 es la menor de todas, a pesar de que esperaría una gran diferencia semántica entre una frase sobre el clima y otra sobre un coche. Esto sugiere que tal vez el modelo no está captando completamente las diferencias temáticas que yo percibo.  


Estos resultados implican varias cosas:  


**Calibración del modelo:** Es posible que el modelo bert-base-uncased que estoy utilizando no esté completamente ajustado para captar todas las sutilezas del contexto en este experimento.


**Sensibilidad y variabilidad del modelo:** Los modelos de lenguaje como BERT son sensibles al contexto y las palabras utilizadas, y pequeñas variaciones en el texto pueden generar cambios significativos en los embeddings. Además, la variabilidad en las representaciones podría estar afectando los resultados.  


En resumen, aunque los resultados muestran que el modelo tiene cierta capacidad para discernir similitudes y diferencias semánticas, también indican que puede haber limitaciones o aspectos peculiares en cómo se están calculando o interpretando estos embeddings. Sigo teniendo el "problema" que los chuncks comienzan por palabras ya empezadas.